# Whatsapp Web Scraper Using Selenium

This application will scrape whatsapp web using selenium

In [42]:
import configparser
import time

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

In [43]:
driver = webdriver.Chrome("./chromedriver.exe")
driver.get("http://web.whatsapp.com")

In [46]:
# get the name of all the chat contacts
for chatter in driver.find_elements_by_xpath("//div[@class='X7YrQ']"):
    chatter_name = chatter.find_element_by_xpath(".//span[contains(@class, '_19RFN')]").text
    print(chatter_name)
    
    # now we get the information in a specific chat
    chatter.find_element_by_xpath(".//div[contains(@class,'_2UaNq')]").click()
    chat_section = driver.find_element_by_xpath(".//div[contains(@class, '_1ays2')]")
    
    # scroll until the start of conversation
    while not driver.find_elements_by_xpath(".//span[@data-icon='lock-small']"):
        chat_section.send_keys(Keys.CONTROL + Keys.HOME)
        time.sleep(2)
        
    # create text_file to save messages
    chat_file = open("wa_chats/convo-{}.txt".format(chatter_name), 'w+')

    # grab all the messages a
    for messages in driver.find_elements_by_xpath(
        "//div[contains(@class,'message-in')] | //div[contains(@class,'message-out')]"):
        
        # get the metadata of the chat i.e date time and username
        message_metadata = messages.find_element_by_xpath("//div[@class='copyable-text']")
        message_metadata = message_metadata.get_attribute("data-pre-plain-text")
        
        # get message text and emojis
        try:
            message = ""
            emojis = []

            message_container = messages.find_element_by_xpath(
                ".//div[@class='copyable-text']")

            message = message_container.find_element_by_xpath(
                ".//span[contains(@class,'selectable-text invisible-space copyable-text')]"
            ).text

            for emoji in message_container.find_elements_by_xpath(
                ".//img[contains(@class,'selectable-text invisible-space copyable-text')]"
            ):
                emojis.append(emoji.get_attribute("data-plain-text"))

        except NoSuchElementException:  # In case there are only emojis in the message
            try:
                message = ""
                emojis = []
                message_container = messages.find_element_by_xpath(
                    ".//div[@class='copyable-text']")

                for emoji in message_container.find_elements_by_xpath(
                        ".//img[contains(@class,'selectable-text invisible-space copyable-text')]"
                ):
                    emojis.append(emoji.get_attribute("data-plain-text"))
            except NoSuchElementException:
                pass
            
            
        # now, to format the message into a similar style as a whatsapp exported chat
        # format follows the following: dd/mm/yyyy, hh:mm - [username]: [chat_message]
        
        message_datetime = message_metadata.replace("]", " -").strip("[")
        msg = message_datetime + message + "\n"
        chat_file.write(msg)
        
    chat_file.close()
        

SyntaxError: invalid syntax (<ipython-input-46-a5d87e8a935c>, line 62)